In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100

# A GenRP tutorial

First generate a simulated dataset:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

t = np.sort(np.random.uniform(0, 10, 147))  # The input coordinates must be sorted
yerr = np.random.uniform(0.08, 0.22, len(t))
y = 0.2 * (t-5) + np.sin(3*t + 0.1*(t-5)**2) + yerr * np.random.randn(len(t))

plt.errorbar(t, y, yerr=yerr, fmt=".k", capsize=0)
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0, 10)
plt.ylim(-2.5, 2.5);

Now let's fit this using `genrp`.

In [ ]:
import genrp
from genrp import kernels

# A periodic component
Q = 1.0
w0 = 3.0
S0 = np.var(y) / (w0 * Q)
kernel = kernels.SHOTerm(np.log(S0), np.log(Q), np.log(w0))

# A non-periodic component
Q = 1.0 / np.sqrt(2.0)
w0 = 3.0
S0 = np.var(y) / (w0 * Q)
kernel += kernels.SHOTerm(np.log(S0), np.log(Q), np.log(w0))

gp = genrp.GP(kernel)
gp.compute(t, yerr)
print("Initial log likelihood: {0}".format(gp.log_likelihood(y)))

x = np.linspace(0, 10, 5000)
initial_mu, initial_var = gp.predict(y, x, return_var=True)

initial_color = "#1f77b4"
plt.errorbar(t, y, yerr=yerr, fmt=".k", capsize=0)
plt.plot(x, initial_mu, color=initial_color)
plt.fill_between(x, initial_mu + np.sqrt(initial_var),
                 initial_mu - np.sqrt(initial_var),
                 color=initial_color, alpha=0.3)
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0, 10)
plt.ylim(-2.5, 2.5)

plt.figure()
omega = np.linspace(0.1, 10, 5000)
initial_psd = gp.kernel.get_psd(omega)
plt.plot(omega, initial_psd, color=initial_color)
plt.yscale("log")
plt.xscale("log")
plt.xlim(1, 10)
plt.xlabel("$\omega$")
plt.ylabel("$S(\omega)$");

We can look at the parameters:

In [ ]:
gp.get_parameter_dict()

Let's freeze the Q-factor for the non-periodic component so that we don't fit for it:

In [ ]:
gp.freeze_parameter("kernel:k2:log_Q")
gp.get_parameter_dict()

Fit for the maximum likelihood parameters:

In [ ]:
from scipy.optimize import minimize

def neg_log_like(params):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y)

initial_params = gp.get_parameter_vector()
bounds = [(-15, 15) for _ in range(len(initial_params))]

r = minimize(neg_log_like, initial_params, method="L-BFGS-B", bounds=bounds)
gp.set_parameter_vector(r.x)
print(r)

Here are the maximum likelihood parameters:

In [ ]:
gp.get_parameter_dict()

And here are the predictions that the maximum likelihood model makes:

In [ ]:
mu, var = gp.predict(y, x, return_var=True)

ml_color = "#ff7f0e"
plt.errorbar(t, y, yerr=yerr, fmt=".k", capsize=0)
plt.plot(x, initial_mu, color=initial_color)
plt.fill_between(x, initial_mu + np.sqrt(initial_var),
                 initial_mu - np.sqrt(initial_var),
                 color=initial_color, alpha=0.3)
plt.plot(x, mu, color=ml_color)
plt.fill_between(x, mu + np.sqrt(var), mu - np.sqrt(var), color=ml_color, alpha=0.3)
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0, 10)
plt.ylim(-2.5, 2.5)

plt.figure()
psd = gp.kernel.get_psd(omega)
plt.plot(omega, initial_psd, color=initial_color, label="initial")
plt.plot(omega, psd, color=ml_color, label="max likelihood")
plt.legend(loc=3, fontsize=12)

# Let's plot each component of the PSD:
for k in gp.kernel.terms:
    plt.plot(omega, k.get_psd(omega), "--", color=ml_color)

plt.yscale("log")
plt.xscale("log")
plt.xlim(omega[0], omega[-1])
plt.xlabel("$\omega$")
plt.ylabel("$S(\omega)$");